# E Certificate generator & Sender Script

- This script will generate e Certificates for all the participants who have registered and attended the respective events.
- A certificate template is created in any of the graphic design softwares, and a png image is obtained. The px coordinate of the information section is taken with help of photoshop or ms paint, by simply hovering the mouse over that place.
- The participants data along with their contact information and attendance are obtained by respective head volunteers, which is cleaned up before exporting into a csv file.
- All the necessary files are first placed into a new directory, and within that directory, separate directories for each event is created.
- The script will be then executed, which will create a subdirectory of email addresses, containing the generated eCertificates.
- The next part of the script will take the subdirectory's name, which holds the email address, and  sends the generated eCertificates and a html template containing a greeting to each of the participating teams.

### 1. Importing necessary libraries

In [1]:
import pandas as pd
import cv2
import os
import time
from pathlib import Path

import base64
from email.message import EmailMessage

import smtplib
from email.utils import formataddr
from email import encoders
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart

from PIL import Image
import img2pdf

### 2. Previewing the database

In [2]:
event = 'robotrack'
df = pd.read_csv(f'./Technotsav/Data/{event}.csv')
df

,name,attendance,univ,p2,p3,email
0,A********* H********* No,B********* I********* of Technology,M********* A********* G********* REDACTED@gma...,NaN,NaN,NaN
1,Khivan,Yes,S********* M********* V********* I********* of...,P********* a,D********* hb,REDACTED@gmail.com
2,M********* A********* moez,Yes,A********* U********* F********* mohammedi,N********* ahmed khan,REDACTED@gmail.com,NaN
3,S********* AS,Yes,A********* E********* College,B********* R********* Reddy,C********* K********* S,REDACTED@gmail.com
4,T********* P********* Yes,B********* I********* of Technology,S********* S********* H********* T,H********* K********* V,REDACTED@gmail.com,NaN
5,T********* B********* G,No,D********* S********* A********* O********* T*...,R********* TR,Yashaswini M A,REDACTED@gmail.com
6,T********* chandar R********* Yes,S********* M. V********* institute of Technology,S********* j jadhav,NaN,REDACTED@gmail.com,NaN
7,V********* N,No,D********* S********* academy of technology an...,S********* H********* S,N********* K********* S,REDACTED@gmail.com
8,V********* hippargi,Yes,S********* M. V********* institute of Technology,D********* M,NaN,REDACTED@gmail.com
9,R********* Tiwari,Yes,S********* M. V********* institute of Technology,M********* Yadu,K********* K********* REDACTED@gmail.com,NaN


### 3. Necessary parameters

In [3]:
name_coordinates = (781,770) #Bottom left corner of text placement
univ_coordinates = (400,840)
font = cv2.FONT_HERSHEY_COMPLEX
univ_fontscale = 1.5
name_fontscale = 2

### 4. Certificate generation

In [4]:
def generate_cert(template, name, name_fontscale, name_coordinates, univ, univ_fontscale, univ_coordinates, destination, thickness=3, name_font = cv2.FONT_HERSHEY_DUPLEX, univ_font = cv2.FONT_HERSHEY_DUPLEX):
    templ = cv2.imread(template)

    cv2.putText(img=templ,text=name, org=name_coordinates, fontFace=name_font, fontScale=name_fontscale, color=(0,0,0), thickness=thickness)
    cv2.putText(templ,univ, univ_coordinates, univ_font, univ_fontscale, (0,0,0), thickness=thickness)

    cv2.imwrite(destination,templ)

    with Image.open(destination) as image:
        pdf = img2pdf.convert(image.filename)

        with open(destination[:-4]+'.pdf', 'wb') as file:
          file.write(pdf)


In [5]:
no_email = [] # List containing names of participants, who did not provide their email address
event = event.lower()
cert_root_folder = f'./Technotsav/Certificates/{event}'
if not os.path.exists(cert_root_folder):
    os.makedirs(cert_root_folder)


for i in range(len(df)):
    if df.loc[i,'attendance'] == 'No':
        continue

    if df.loc[i,'email'] == 'nan':
        no_email.append(df.loc[i,'name'])
        continue

    cert_email_folder = cert_root_folder + f"/{df.loc[i,'email']}"
    if not os.path.exists(cert_email_folder):
        os.makedirs(cert_email_folder)
    

    template = f'./Technotsav/Templates/{event}.png'
    name = df.loc[i,'name']
    univ = df.loc[i,'univ']
    destination = cert_email_folder + f'/{name}.jpg'

    generate_cert(template=template, destination=destination, name=name, name_fontscale=name_fontscale, univ_fontscale=univ_fontscale, univ=univ , univ_coordinates=univ_coordinates, name_coordinates=name_coordinates)


    
    for j in ['p2', 'p3']:

        p_name = str(df.loc[i,j])

        if p_name == 'nan':
            continue

        dest = cert_email_folder + f'/{p_name}.jpg'
        
        generate_cert(template=template, destination=dest, name=p_name, name_fontscale=name_fontscale, univ_fontscale=univ_fontscale, univ=univ, univ_coordinates=univ_coordinates, name_coordinates=name_coordinates)

    for item in os.listdir(cert_email_folder):
        if item.endswith('.jpg'):
            os.remove(os.path.join(cert_email_folder, item))


##### Following code block extracts the list of email address from the subdirectories, opens first email folder and prints the contents for verification.

In [7]:
for root, dirs, files in os.walk(cert_root_folder):
    email_dir = dirs
    break 

for file in os.listdir(cert_root_folder + '/' + email_dir[0]):
    print(file)

# print(email_dir)

M***** M****** M G A.pdf
V**** M V.pdf
S****** K.pdf


### 5. Sending email containing the certificates to Team Leader

Note: enable less secure apps in google accounts settings, else the script will raise an exception stating that bad password was fed.

In [8]:
sender_email = 'sb.ieee@sirmvit.edu'
# receiver_email = ['ch374nvj@gmail.com']
receiver_email = email_dir

for recr in receiver_email:
    if recr == 'nan':
        continue
    message = MIMEMultipart('alternative')
    message['Subject'] = 'technotsav 2k24 Participation Certificates'
    message['From'] = 'IEEE SB SirMVIT <'+sender_email+'>'
    # message['To'] = 'ch374nvj@gmail.com'
    message['To'] = recr
    # message['Bcc'] = sender_email

    for file in os.listdir(cert_root_folder + '/' + recr):
        with open(cert_root_folder + '/' + recr +'/' +file, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename= {file}',
            )
            message.attach(part)


    file1 = open('./Technotsav/Email Template/index.html','r')
    html = file1.read() 
    part1 = MIMEText(html, 'html')
    message.attach(part1)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login('sb.ieee@sirmvit.edu', 'REDACTED')

    sent = False
    while not sent:
        try:
            print('Sending Mail to ', recr)
            s.sendmail(sender_email, recr, message.as_string())
        except Exception as e:
            print("Error occured: ", e)
            print("Trying again...")
            time.sleep(3)
        else:
            sent = True

    print('Sent')

s.quit()

Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@sairamtap.edu.in .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent
Sending REDACTED@gmail.com .....Sent


(221,
 b'2.0.0 closing connection d2e1a72fcca58-705ccb3c2bbsm8938330b3a.98 - gsmtp')